<a href="https://colab.research.google.com/github/fleminjohnson/Multiclass_Multilabel_prediction_For_stack_overflow_Questions/blob/main/Multiclass_Multilabel_prediction_For_stack_overflow_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q --show-progress "https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0"

stacksample.zip?dl= 100%[===================>]   1.15G  52.3MB/s    in 25s     


In [2]:
!unzip /content/stacksample.zip?dl=0

Archive:  /content/stacksample.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [3]:
!rm /content/stacksample.zip?dl=0

In [122]:
import pandas as pd

In [123]:
answers = pd.read_csv("/content/Answers.csv", encoding='latin-1')
questions = pd.read_csv("/content/Questions.csv", encoding='latin-1')
tags = pd.read_csv("/content/Tags.csv", encoding='latin-1')

In [124]:
tags[tags['Id']==90]

,Id,Tag
3,90,svn
4,90,tortoisesvn
5,90,branch
6,90,branching-and-merging


In [125]:
print(questions.head())
print(answers.head())
print(tags.head())

    Id  OwnerUserId          CreationDate            ClosedDate  Score  \
0   80         26.0  2008-08-01T13:57:07Z                   NaN     26   
1   90         58.0  2008-08-01T14:41:24Z  2012-12-26T03:45:49Z    144   
2  120         83.0  2008-08-01T15:50:08Z                   NaN     21   
3  180    2089740.0  2008-08-01T18:42:19Z                   NaN     53   
4  260         91.0  2008-08-01T23:22:08Z                   NaN     49   

                                               Title  \
0  SQLStatement.execute() - multiple queries in o...   
1  Good branching and merging tutorials for Torto...   
2                                  ASP.NET Site Maps   
3                 Function for creating color wheels   
4  Adding scripting functionality to .NET applica...   

                                                Body  
0  <p>I've written a database generation script i...  
1  <p>Are there any really good tutorials explain...  
2  <p>Has anyone got experience creating <strong>... 

In [126]:
print(questions.isnull().sum())
print(answers.isnull().sum())
print(tags.isnull().sum())

Id                    0
OwnerUserId       14454
CreationDate          0
ClosedDate      1208257
Score                 0
Title                 0
Body                  0
dtype: int64
Id                  0
OwnerUserId     13200
CreationDate        0
ParentId            0
Score               0
Body                0
dtype: int64
Id        0
Tag    1113
dtype: int64


In [127]:
tags_count = tags["Tag"].value_counts()
top_10_tags = tags_count.head(10)

In [128]:
tags_count

,count
Tag,
javascript,124155
java,115212
c#,101186
php,98808
android,90659
...,...
ats,1
sn,1
zero-initialization,1


In [129]:
top_10_tags.shape

(10,)

In [130]:
df_tags_filtered = tags[tags['Tag'].isin(top_10_tags.index)]

In [131]:
df_merged = questions.merge(df_tags_filtered, on = 'Id', how = 'inner')

In [132]:
#df_merged

In [133]:
df_tags = df_merged.groupby('Id')['Tag'].apply(list).reset_index()

In [134]:
df_tags

,Id,Tag
0,260,[c#]
1,330,[c++]
2,650,[c#]
3,930,[c#]
4,1010,[c#]
...,...,...
706331,40143150,[javascript]
706332,40143190,[python]
706333,40143210,[php]
706334,40143340,[android]


In [135]:
from sklearn.preprocessing import MultiLabelBinarizer

In [136]:
mlb = MultiLabelBinarizer()
tag_matrix = mlb.fit_transform(df_tags['Tag'])
tag_df = pd.DataFrame(tag_matrix, columns = mlb.classes_)

In [137]:
tag_df.insert(0, "Id", df_tags["Id"])

In [138]:
#merge with question dataset
df = df_merged.merge(tag_df, on="Id", how="inner")

In [139]:
df.drop(['OwnerUserId', 'ClosedDate', 'Score','CreationDate','Tag'], axis=1, inplace=True)  # Keep only relevant features

In [140]:
df

,Id,Title,Body,android,c#,c++,html,ios,java,javascript,jquery,php,python
0,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,0,1,0,0,0,0,0,0,0,0
1,330,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,0,0,1,0,0,0,0,0,0,0
2,650,Automatically update version number,<p>I would like the version property of my app...,0,1,0,0,0,0,0,0,0,0
3,930,How do I connect to a database and loop over a...,<p>What's the simplest way to connect and quer...,0,1,0,0,0,0,0,0,0,0
4,1010,"How to get the value of built, encoded ViewState?",<p>I need to grab the base64-encoded represent...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
826734,40143150,"YouTube iFrame API: no ready call, no error call","<p>I'm trying to detect the ""flash out of date...",0,0,0,0,0,0,1,0,0,0
826735,40143190,How to execute multiline python code from a ba...,<p>I need to extend a shell script (bash). As ...,0,0,0,0,0,0,0,0,0,1
826736,40143210,URL routing in PHP (MVC),<p>I am building a custom MVC project and I ha...,0,0,0,0,0,0,0,0,1,0
826737,40143340,Obfuscating code in android studio,<p>Under minifyEnabled I changed from false to...,1,0,0,0,0,0,0,0,0,0


In [141]:
df['Title'] = df['Title'].str.lower()
df['Body'] = df['Body'].str.lower()
df['Body'] = df['Body'] + " " + df['Title']
df['Body'] = df['Body'].str.replace(r'<[^<>]*>', '', regex=True)

In [142]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [143]:
MAX_NUM_WORDS = 50000
MAX_SEQUENCE_LENGTH = 300

In [144]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(df["Body"])

import pickle

# Save tokenizer
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [145]:
# import pickle
# with open("tokenizer.pickle", "rb") as handle:
#     tokenizer = pickle.load(handle)

In [146]:
X = tokenizer.texts_to_sequences(df["Body"])

In [147]:
# Load sequences
#X = np.load("X_sequences.npy", allow_pickle=True)

In [148]:
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")

In [149]:
from sklearn.model_selection import train_test_split

In [150]:
top_10_tags

,count
Tag,
javascript,124155
java,115212
c#,101186
php,98808
android,90659
jquery,78542
python,64601
html,58976
c++,47591


In [151]:
Y = df[top_10_tags.index]

In [152]:
X_train,X_val,Y_train,Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [153]:
print("Shape of Training Data:", X_train.shape, Y_train.shape)
print("Shape of Validation Data:", X_val.shape, Y_val.shape)

Shape of Training Data: (661391, 300) (661391, 10)
Shape of Validation Data: (165348, 300) (165348, 10)


In [154]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, Flatten
from tensorflow.keras.regularizers import l2

In [155]:
# Hyperparameters
EMBEDDING_DIM = 100  # Each word will be represented by a 100-dimensional vector

#from tensorflow.keras.layers import GlobalAveragePooling1D

from tensorflow.keras.layers import LSTM, Bidirectional

model = Sequential([
    Embedding(input_dim=50000, output_dim=EMBEDDING_DIM, input_length=300),
    LSTM(128),
    Dropout(0.7),
    Dense(10, activation='sigmoid')
])


from tensorflow.keras.optimizers import Adam

# Use a smaller learning rate (default Adam LR is 0.001)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

# Model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [156]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)

history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=32,
    epochs=40,  # Start with 10 epochs (can be increased later)
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)


Epoch 1/40
20669/20669 ━━━━━━━━━━━━━━━━━━━━ 322s 15ms/step - accuracy: 0.4939 - loss: 0.2607 - val_accuracy: 0.7706 - val_loss: 0.0997 - learning_rate: 0.0010
Epoch 2/40
20669/20669 ━━━━━━━━━━━━━━━━━━━━ 320s 15ms/step - accuracy: 0.7752 - loss: 0.1063 - val_accuracy: 0.7798 - val_loss: 0.0908 - learning_rate: 0.0010
Epoch 3/40
20669/20669 ━━━━━━━━━━━━━━━━━━━━ 327s 16ms/step - accuracy: 0.7859 - loss: 0.0927 - val_accuracy: 0.7772 - val_loss: 0.0884 - learning_rate: 0.0010
Epoch 4/40
20669/20669 ━━━━━━━━━━━━━━━━━━━━ 325s 16ms/step - accuracy: 0.7930 - loss: 0.0829 - val_accuracy: 0.7731 - val_loss: 0.0873 - learning_rate: 0.0010
Epoch 5/40
20669/20669 ━━━━━━━━━━━━━━━━━━━━ 324s 16ms/step - accuracy: 0.7964 - loss: 0.0745 - val_accuracy: 0.7730 - val_loss: 0.0885 - learning_rate: 0.0010
Epoch 6/40
20669/20669 ━━━━━━━━━━━━━━━━━━━━ 327s 16ms/step - accuracy: 0.8003 - loss: 0.0677 - val_accuracy: 0.7670 - val_loss: 0.0878 - learning_rate: 0.0010
Epoch 7/40
20669/20669 ━━━━━━━━━━━━━━━━━━━━ 32

In [157]:
predictions = model.predict(X_val)

5168/5168 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step


In [158]:
predictions[predictions >= 0.5] = 1
predictions[predictions < 0.5] = 0

In [159]:
loss, accuracy = model.evaluate(X_val, Y_val)
print(f"Validation Loss: {loss}, Accuracy: {accuracy}")

5168/5168 ━━━━━━━━━━━━━━━━━━━━ 32s 6ms/step - accuracy: 0.7744 - loss: 0.0867
Validation Loss: 0.08729509264230728, Accuracy: 0.7731027603149414


In [160]:
for i in range(10):  # Check the first 10 validation examples
    true_tags = mlb.inverse_transform(Y_val.iloc[i:i+1].to_numpy())[0]  # Convert to NumPy array
    predicted_tags = mlb.inverse_transform(predictions[i:i+1])[0]  # Use prediction matrix

    # Print question (assuming you stored original text before tokenization)
    print(f"\nExample {i+1}:")
    print(f"  Question: {df['Body'].iloc[i][:100]}...")  # Use original text before tokenization
    print(f"  True Tags: {true_tags}")
    print(f"  Predicted Tags: {predicted_tags}")



Example 1:
  Question: i have a little game written in c#. it uses a database as back-end. it's 
a trading card game, and i...
  True Tags: ('ios',)
  Predicted Tags: ('ios',)

Example 2:
  Question: i am working on a collection of classes used for video playback and recording. i have one main class...
  True Tags: ('jquery',)
  Predicted Tags: ('jquery',)

Example 3:
  Question: i would like the version property of my application to be incremented for each build but i'm not sur...
  True Tags: ('c#',)
  Predicted Tags: ('c#',)

Example 4:
  Question: what's the simplest way to connect and query a database for a set of records in c#?
 how do i connec...
  True Tags: ('android',)
  Predicted Tags: ('android',)

Example 5:
  Question: i need to grab the base64-encoded representation of the viewstate. obviously this would not be avail...
  True Tags: ('ios',)
  Predicted Tags: ('ios',)

Example 6:
  Question: i'm looking for a way to delete a file which is locked by another process using